In [1]:
import json
import os
import shutil
import uuid
from datetime import date, timedelta
from pathlib import Path
from typing import Any, Iterator

import uvicorn
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from dotenv import load_dotenv
from fastapi import FastAPI, HTTPException
from fastapi.responses import FileResponse, StreamingResponse
from fastapi.staticfiles import StaticFiles
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

/Users/felixcxr/code/wanxiao_demo/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PROJECT_ROOT = Path.cwd()
FRONTEND_DIR = PROJECT_ROOT / "frontend"
MEMORIES_DIR = PROJECT_ROOT / "memories"
DAILY_DIR = MEMORIES_DIR / "daily"
LONG_TERM_FILE = MEMORIES_DIR / "MEMORY.md"

load_dotenv()

model_name = os.getenv("OPENROUTER_MODEL", "z-ai/glm-4.7-flash")


In [3]:
def _ensure_memory_files(today: date) -> tuple[str, str]:
    MEMORIES_DIR.mkdir(parents=True, exist_ok=True)
    DAILY_DIR.mkdir(parents=True, exist_ok=True)

    today_name = today.strftime("%Y-%m-%d")
    today_file = DAILY_DIR / f"{today_name}.md"

    if not LONG_TERM_FILE.exists():
        LONG_TERM_FILE.write_text(
            "# 长期记忆\n\n"
            "## 用户偏好\n"
            "- 暂无\n\n"
            "## 重要决策\n"
            "- 暂无\n\n"
            "## 关键联系人\n"
            "- 暂无\n\n"
            "## 项目事实\n"
            "- 暂无\n",
            encoding="utf-8",
        )

    if not today_file.exists():
        today_file.write_text(
            f"# {today_name}\n\n"
            "## 09:00 - 会话初始化\n"
            "- 新的一天开始，按需记录重要事实、决策、偏好与待办。\n",
            encoding="utf-8",
        )

    return "/memories/MEMORY.md", f"/memories/daily/{today_name}.md"


In [23]:
external_skills_dir = Path.home() / ".deepagents" / "agent" / "skills"
mirrored_skills_dir = PROJECT_ROOT / "skills"

In [24]:
_sync_skills_to_project(external_skills_dir, mirrored_skills_dir)

In [25]:
print(mirrored_skills_dir)

/Users/felixcxr/code/wanxiao_demo/skills


In [20]:
def build_agent() -> Any:
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        raise RuntimeError("OPENROUTER_API_KEY is missing. Please set it in .env")

    os.environ["OPENAI_API_KEY"] = api_key

    llm = ChatOpenAI(
        model=model_name,
        base_url="https://openrouter.ai/api/v1",
        # temperature=0.2,
    )

    external_skills_dir = Path.home() / ".deepagents" / "agent" / "skills"
    mirrored_skills_dir = PROJECT_ROOT / "skills"
    _sync_skills_to_project(external_skills_dir, mirrored_skills_dir)
    today = date.today()
    yesterday = today - timedelta(days=1)
    long_term_path, today_path = _ensure_memory_files(today)
    yesterday_path = f"/memories/daily/{yesterday.strftime('%Y-%m-%d')}.md"

    system_prompt = (
        "你是一个通用 DeepAgent。你需要使用双层记忆系统，并把记忆写入本地 Markdown 文件。\n\n"
        "记忆目录结构：\n"
        "- 长期记忆：/memories/MEMORY.md\n"
        "- 每日日志：/memories/daily/YYYY-MM-DD.md\n\n"
        "写入规范（参考双层记忆）：\n"
        "A. 每日日志（短期记忆）\n"
        "- 仅追加写入到当天文件 /memories/daily/YYYY-MM-DD.md。\n"
        "- 使用格式：\n"
        "  ## HH:MM - 事件标题\n"
        "  - 事实/结论1\n"
        "  - 事实/结论2\n"
        "- 适合记录：会话过程、临时上下文、待办进展、当天细节。\n\n"
        "B. 长期记忆\n"
        "- 维护 /memories/MEMORY.md 的结构化内容，优先保持以下章节：\n"
        "  ## 用户偏好\n"
        "  ## 重要决策\n"
        "  ## 关键联系人\n"
        "  ## 项目事实\n"
        "- 适合记录：跨天仍有效的信息（稳定偏好、关键决策、固定事实）。\n"
        "- 写入时去重、合并相同信息，避免重复条目。\n\n"
        "执行原则：\n"
        "- 当用户明确说‘记住这个’或出现重要信息时，优先写每日日志，并在必要时同步更新长期记忆。\n"
        "- 如果信息不确定，请标注‘待确认’，不要把猜测写成事实。\n"
        "- 回答用户问题时可结合记忆文件，但输出给用户时保持简洁准确。\n"
    )
    print(mirrored_skills_dir)
    agent = create_deep_agent(
        model=llm,
        backend=FilesystemBackend(root_dir=str(PROJECT_ROOT), virtual_mode=True),
        skills=[str(mirrored_skills_dir)],
        memory=[long_term_path, today_path],
        system_prompt=system_prompt,
    )
    return agent


In [21]:
def get_agent() -> Any:
    global AGENT
    if AGENT is None:
        AGENT = build_agent()
    return AGENT

In [22]:
agent = get_agent()

In [6]:
def _to_deepagent_messages(history: list[Any], user_text: str) -> list[dict[str, str]]:
    messages: list[dict[str, str]] = []
    for item in history:
        if isinstance(item, dict):
            role = item.get("role")
            content = item.get("content", "")
            if role in {"user", "assistant"}:
                messages.append({"role": role, "content": str(content)})
            continue

        if isinstance(item, (list, tuple)) and len(item) == 2:
            user_msg, assistant_msg = item
            if user_msg:
                messages.append({"role": "user", "content": str(user_msg)})
            if assistant_msg:
                messages.append({"role": "assistant", "content": str(assistant_msg)})

    messages.append({"role": "user", "content": user_text})
    return messages


In [7]:
def _sync_skills_to_project(source_dir: Path, target_dir: Path) -> None:
    target_dir.mkdir(parents=True, exist_ok=True)
    if not source_dir.exists():
        return

    for root, dirs, files in os.walk(source_dir):
        rel_root = Path(root).relative_to(source_dir)

        dirs[:] = [d for d in dirs if d != "__pycache__"]
        if any(part == "__pycache__" for part in rel_root.parts):
            continue

        for file_name in files:
            src = Path(root) / file_name
            if src.suffix == ".pyc":
                continue

            dst = target_dir / rel_root / file_name
            dst.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(src, dst)

In [13]:
def chat_with_agent(
    message: str, history: list[Any], thread_id: str | None
) -> tuple[str, str, list[dict[str, str]]]:
    resolved_thread_id = thread_id or str(uuid.uuid4())
    normalized_messages = _to_deepagent_messages(history, message)

    result = get_agent().invoke(
        {"messages": normalized_messages},
        config={"configurable": {"thread_id": resolved_thread_id}},
    )

    assistant_text = ""
    for msg in reversed(result.get("messages", [])):
        if _get_msg_type(msg) == "ai":
            assistant_text = _message_content_to_text(_get_msg_content(msg))
            break

    updated_history = list(normalized_messages)
    updated_history.append({"role": "assistant", "content": assistant_text})
    return assistant_text, resolved_thread_id, updated_history

In [14]:
def _message_content_to_text(content: Any) -> str:
    if isinstance(content, str):
        return content

    if isinstance(content, list):
        parts: list[str] = []
        for item in content:
            if isinstance(item, dict):
                text = item.get("text") or item.get("content")
                if isinstance(text, str):
                    parts.append(text)
            elif hasattr(item, "text"):
                text = getattr(item, "text", "")
                if isinstance(text, str):
                    parts.append(text)
            elif hasattr(item, "content"):
                text = getattr(item, "content", "")
                if isinstance(text, str):
                    parts.append(text)
            elif isinstance(item, str):
                parts.append(item)
        return "".join(parts)

    return ""


In [9]:
def _get_msg_type(msg: Any) -> str:
    if isinstance(msg, dict):
        return str(msg.get("type", ""))
    return str(getattr(msg, "type", ""))


def _get_msg_content(msg: Any) -> Any:
    if isinstance(msg, dict):
        return msg.get("content", "")
    return getattr(msg, "content", "")


def _get_tool_calls(msg: Any) -> list[Any]:
    if isinstance(msg, dict):
        tool_calls = msg.get("tool_calls")
        return tool_calls if isinstance(tool_calls, list) else []
    tool_calls = getattr(msg, "tool_calls", None)
    return tool_calls if isinstance(tool_calls, list) else []

In [15]:
AGENT: Any | None = None

In [16]:
result = chat_with_agent("你是什么模型",[],None)

In [17]:
print(result)

('我是 DeepAgent，一个基于双层记忆系统的智能助手。\n\n## 我的核心能力：\n\n**1. 双层记忆系统**\n- 长期记忆：存储你的偏好、重要决策、固定事实（/memories/MEMORY.md）\n- 每日日志：记录当天详细过程和进展（/memories/daily/YYYY-MM-DD.md）\n\n**2. 文件与代码操作**\n- 读写、编辑、搜索文件\n- 执行脚本和代码分析\n- 使用 glob/grep 等模式匹配工具\n\n**3. 任务规划**\n- 创建结构化的待办清单\n- 跟踪多步骤任务的进展\n\n**4. 子代理系统**\n- 可以启动专门的子代理处理复杂任务\n- 支持并行执行多个独立任务\n\n**5. Skills 系统**\n- 内置专门领域的技能库\n- 渐进式披露，按需激活使用\n\n你可以让我帮你做些什么？比如文件处理、代码分析、任务规划、研究查询等。', '10eaff9d-be10-465a-bef8-ce9e468e5558', [{'role': 'user', 'content': '你是什么模型'}, {'role': 'assistant', 'content': '我是 DeepAgent，一个基于双层记忆系统的智能助手。\n\n## 我的核心能力：\n\n**1. 双层记忆系统**\n- 长期记忆：存储你的偏好、重要决策、固定事实（/memories/MEMORY.md）\n- 每日日志：记录当天详细过程和进展（/memories/daily/YYYY-MM-DD.md）\n\n**2. 文件与代码操作**\n- 读写、编辑、搜索文件\n- 执行脚本和代码分析\n- 使用 glob/grep 等模式匹配工具\n\n**3. 任务规划**\n- 创建结构化的待办清单\n- 跟踪多步骤任务的进展\n\n**4. 子代理系统**\n- 可以启动专门的子代理处理复杂任务\n- 支持并行执行多个独立任务\n\n**5. Skills 系统**\n- 内置专门领域的技能库\n- 渐进式披露，按需激活使用\n\n你可以让我帮你做些什么？比如文件处理、代码分析、任务规划、研究查询等。'}])


In [26]:
from pathlib import Path
from typing import Any


# =============================================================================
# 配置和数据加载
# =============================================================================

CUSTOMER_CSV_PATH = Path("/home/daytona/data/customer_db.csv")
